## Create Capacity

Fabric Capacity is an Azure Resource that is manipulated via the Azure Management API (management.azure.com).  In order to create the capacity it requires basic information like name, location, SKU, and one or more administrators.

### Notes:
1. Constants this notebook depends on are set globally in the [/services/env.py](/services/env.py) class file.
1. This API is not intelligent enough to create the capacity if it does not exist, so it assumes the capacity already exists, if a matching capacity does not exist an error will be thrown.
1. If you're working through the notebooks in this sample, this notebook is simply changing the size of the capacity from an F2 to an F4.
1. I was unable to find documentation for this specific resource type.  I reverse engineered this call by using the Azure Portal and choosing 'Download a Template' option during resoruce creation.

In [ ]:
pip install requests msal

In [ ]:
#For orgaization purposes I put notbooks in subfolders not the root of the proejct.aad_token
#This code adds the root directory of the project to the sys path so we can load class modules from the services folder
#I think this only needs to be run once, but including it for completeness.
import os, sys
projectRoot = os.path.abspath('.')
directory = os.path.dirname(projectRoot)
if not directory in sys.path: sys.path.append(directory)

In [ ]:
#This leverages the code encapsulated in services/aadservice.py that encapsulates the service principle login.
#Note: for most of the other notebooks we're not passing in a scope, this is because the code in aadservice.property
#defaults the scope to a scope for the Power BI API.   Since we're calling the azure management API we need a different
#scope.
from services.aadservice import AadService
scope = 'https://management.azure.com/.default'
aadToken = AadService.get_access_token(scope)

headers =  {'Content-Type': 'application/json', 'Authorization': 'Bearer ' + aadToken}

In [ ]:
import requests
import json
from services.env import const

apiUrl = f'https://management.azure.com/subscriptions/{const.subscriptionId}/resourceGroups/{const.resourceGroup}/providers/Microsoft.Fabric/capacities/{const.capacityName}?api-version=2022-07-01-preview'       

body = { 
            "type": "Microsoft.Fabric/capacities",
            "name": const.capacityName,
            "location": "westus3",
            "sku": {
                "name": "F2",
                "tier": "Fabric"
            },
            "properties": {
                "administration": {
                    "members": const.administrators
                }
            }
        }

apiResponse = requests.put(apiUrl, headers=headers, data=json.dumps(body))

#error handling for create capacity
if apiResponse.status_code != 201 and apiResponse.status_code != 200:
    description = f'Error creating capacity:\n  -Status Code:\t{apiResponse.status_code}\n  -Reason:\t{apiResponse.reason}\n  -RequestId:\t{apiResponse.headers.get("RequestId")}\n  -Text:\t{apiResponse.text}'
    print(description)
else:
    apiResponse = json.loads(apiResponse.text)
    print(json.dumps(apiResponse,indent=2))